This notebook shows the code produced to refine the PTB Diagnostic ECG database to provide a smaller subset of the database for which analysis can take place on. Two classes have been defined, a patient class aswell as a class to store each allowed patient.

The data has been filtered based on these factors: 
 - at least ? minutes worth of recording (will cut all files to the first one minutes)
 - access to meta data
 - removed repeated ECGs from the same patient as this could skew results down the line
 
In addition to this the data that will be kept will only consist of the V1-V6 channels because...


In [1]:
import wfdb
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import neurokit2 as nk

First I have got the list of available patients within the desired database.

In [2]:
patients = wfdb.get_record_list('ptbdb') #creates a list containing the directories of all the ECGs within the data base

Creating a class to include the desired information for each patient, with relevant getter and setter functions.

In [3]:
class Patient:
    def __init__(self, name, age, gender, health_state, diagnosis, smoker):
        self._docstring = str
        self._name = str
        self._age = int
        self._gender = str
        self._health_state = str
        self._diagnosis = str
        self._smoker = str
        self._ecg_data = None
        self._signal = None
        self._desired_sample_length = int
        self._channel_names = ["v1", "v2", "v3", "v4", "v5", "v6"]
        
    def set_docstring(self, docstr):
        self._docstring = docstr

    def set_name(self, name):
        self._name = name
        
    def set_health_state(self, health_state):
        self._health_state = health_state

    def set_diagnosis(self, diagnosis):
        self._diagnosis = diagnosis

    def set_ecg_data(self, ecg_data):
        self._ecg_data = ecg_data
        
    def set_signal(self, signal):
        self._signal = signal

    def set_age(self, age):
        self._age = age
        
    def set_gender(self, gender):
        self._gender = gender
    
    def set_smoker_status(self, smoker):
        self._smoker = smoker
        
    def set_channel_names(self, channel_names):
        self._channel_names = channel_names
    
    def set_desired_sample_length(self, desired_sample_length):
        self._desired_sample_length = desired_sample_length
        
    def get_docstring(self):
        return self._docstring
    
    def get_name(self):
        return self._name
    
    def get_age(self):
        return self._age
    
    def get_gender(self):
        return self._gender
    
    def get_smoker_status(self):
        return self._smoker
    
    def get_ecg_data(self):
        return self._ecg_data
    
    def get_diagnosis(self):
        return self._diagnosis   
    
    def get_health_state(self):
        return self._health_state
    
    def get_channel_names(self):
        return self._channel_names
    
    def get_desired_sample_length(self):
        return self._desired_sample_length
    
    def get_info(self):
        info = f"Name: {self._name} \n Age: {self._age}\n Gender: {self._gender}\n"
        info += f"Health State: {self._health_state}\nDiagnosis: {self._diagnosis}\n"
        info += f"ECG Data: {self._ecg_data} \nSmoker: {self._smoker}"
        return info
    
    def plot_ecg(self):
        wfdb.plot_wfdb(record=self.get_ecg_data(), title='All Channels for '+ self._name)
        
    def plot_signal(self):
        wfdb.plot_items(signal = self._signal)
        
    def read_record(self, given_channel_names=None):
        if given_channel_names is not None:
            self.set_channel_names(given_channel_names)
        record = wfdb.rdrecord('C:\\Users\\court\\Documents\\masters\\solo project\\ES98C-ECG-Project\\ptb-diagnostic-ecg-database-1.0.0\\ptb-diagnostic-ecg-database-1.0.0\\' + self.get_docstring(), channel_names = self.get_channel_names(), sampto = self.get_desired_sample_length())
        return record
    
    def read_signal(self, given_channel_names = None):
        if given_channel_names is not None:
            self.set_channel_names(given_channel_names)
        signal, _ = wfdb.rdsamp('C:\\Users\\court\\Documents\\masters\\solo project\\ES98C-ECG-Project\\ptb-diagnostic-ecg-database-1.0.0\\ptb-diagnostic-ecg-database-1.0.0\\' + self.get_docstring(), channel_names = self.get_channel_names(), sampto = self.get_desired_sample_length())
        return signal
    
    def get_signal(self):
        return self._signal
    
    def plot_channels(self, channel_names):
        record = self.read_record(given_channel_names = channel_names)
        wfdb.plot_wfdb(record=record, title='Channel(s) '+ ', '.join(channel_names) + ' for '+ self._name)
        
        
        

Creating a class to contain a list of all allowed patients

In [10]:
class PatientCollection:
    def __init__(self):
        self._patients = []
        self._diagnosis_list = ['Myocardial infarction', 'Healthy control', 'Dysrhythmia', 'Cardiomyopathy', 'Hypertrophy', 'Bundle branch block', 'Valvular heart disease', 'Stable angina', 'Myocarditis']


    def add_patient(self, patient):
        self._patients.append(patient)
        
        
    def get_patients(self, indx):
        #change this too work for multiple patients
        return self._patients[indx] 
    
    
    def get_records(self):
        return [patient.get_ecg_data() for patient in self._patients]
    
    def count_patients(self):
        return len(self._patients)

    def get_all_info(self):
        return [patient.get_info() for patient in self._patients]

    def count_smokers(self):
        return sum(1 for patient in self._patients if patient._smoker == 'yes')
    
    def count_healthy(self):
        return sum(1 for patient in self._patients if patient._health_state == 'Healthy')
    
    def count_unhealthy(self):
        return sum(1 for patient in self._patients if patient._health_state == 'Unhealthy')
    
    def count_gender(self):
        return sum(1 for patient in self._patients if patient._gender == 'Male')
    
    def get_diagnosis_list(self):
        return self._diagnosis_list
    
    def get_diagnoses(self):
        return [patient._health_state for patient in self._patients]
    
    def get_diagnosis_counts(self):
        diagnoses = [patient._diagnosis for patient in self._patients] #creates list of each diagnosis within the patients in the list
        diagnosis_counts = {diagnosis: diagnoses.count(diagnosis) for diagnosis in set(diagnoses)} #counts each diagnosis, set()removes repeats
        return diagnosis_counts
    
    def plot_diagnosis(self):
        diagnosis_counts = self.get_diagnosis_counts()
        fig, axes = plt.subplots(1, 1, figsize = (12, 10))
        
        bars = axes.bar(diagnosis_counts.keys(), diagnosis_counts.values())
        
        # annotate the values on top of each bar
        for bar in bars:
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2.0, yval, int(yval), ha='center', va='bottom')
        axes.set_xlabel('Diagnosis')
        axes.set_ylabel('Count')
        axes.set_title('Diagnosis Distribution')
        axes.legend([f"Total Number of People: {self.count_patients()}, Healthy: {self.count_healthy()}", f"Unhealthy: {self.count_unhealthy()}"])
        plt.show()
        
    def plot_sample_ecg(self):
        plot_indx = np.random.randint(0, self.count_patients())
        patient = self.get_patient(plot_indx)
        patient.plot_ecg()
        
    
        
        

The code below filters the ECGs in the database to the required stringencies

In [11]:
age_meta_idx = 0
smoking_meta_idx = 8
gender_meta_idx = 1
diagnosis_meta_idx = 4
previous_label = 'n/a'

#definite application
duplicate_data_filtering = True
sample_length = 60000


age_data_filtering = True
smoking_data_filtering = False #smoking filter gets rid of alot of healthy patients -leaves only 3 remaining
gender_data_filtering = True

#156 people and 3 healthy if all filters applied

allowed_patients = PatientCollection()

print('Filtering Database')

j=0
for i in tqdm(range(0, len(patients))):
    #creating new patient instance
    new_Patient = Patient(None, None, None, None, None, None)
    new_Patient.set_desired_sample_length(sample_length)
    
    #setting bool to true unless conditions for appending is not met
    append = True
    
    #try and except clause to ensure only ECGs longer than the minimum length are kept
    try:
        record = wfdb.rdrecord('C:\\Users\\court\\Documents\\masters\\solo project\\ES98C-ECG-Project\\ptb-diagnostic-ecg-database-1.0.0\\ptb-diagnostic-ecg-database-1.0.0\\' + patients[i], channel_names = new_Patient.get_channel_names(), sampto = new_Patient.get_desired_sample_length())
        signals, fields = wfdb.rdsamp('C:\\Users\\court\\Documents\\masters\\solo project\\ES98C-ECG-Project\\ptb-diagnostic-ecg-database-1.0.0\\ptb-diagnostic-ecg-database-1.0.0\\' + patients[i], sampto = new_Patient.get_desired_sample_length(), channel_names = new_Patient.get_channel_names())
        
        #append ECG data to Patient instance
        new_Patient.set_ecg_data(record)
        #new_Patient.set_signal(signals)
        
        new_Patient.set_docstring(patients[i])
        
        #labelling the diagnosis within the class
        if record.comments[diagnosis_meta_idx][22:] not in allowed_patients.get_diagnosis_list():
            append = False
        else:
            new_Patient.set_diagnosis(record.comments[diagnosis_meta_idx][22:])
            
        #labelling the health state of each patient class
        if record.comments[diagnosis_meta_idx][22:] == 'Healthy control':
            new_Patient.set_health_state('Healthy')
        else:
            new_Patient.set_health_state('Unhealthy')
        
        if age_data_filtering:
            #making sure age meta data is available
            if record.comments[age_meta_idx] == None:
                append = False
            else:
                new_Patient.set_age(int(record.comments[age_meta_idx][5:]))
                
        if smoking_data_filtering:
            #ensuring smoking meta data is available
            if record.comments[smoking_meta_idx][8:] not in ('yes', 'no'):
                append = False
            else:
                new_Patient.set_smoker_status(record.comments[smoking_meta_idx][8:])
                
        if gender_data_filtering:
            #ensuring gender meta data is available
            if record.comments[gender_meta_idx][5:] not in ('male', 'female'):
                append = False
            else:
                new_Patient.set_gender(record.comments[gender_meta_idx][5:])
            
        if duplicate_data_filtering:
            #only appending the first ECG from each patient to remove duplicates
            patient_number = patients[i][:10]
            if patient_number == previous_label:
                append = False
            else:
                previous_label = patient_number
                new_Patient.set_name(patients[i][:10])
                    
        if append:
#             #removes whole patient if signal quality of one channel is not good enough     
#             signal_quality = []
#             bad_signals = []
#             for j in range(0, len(new_Patient.get_channel_names())):
#                     cleaned_signal = nk.ecg_clean(signals[:, j])
#                     quality = nk.ecg_quality(cleaned_signal, method='zhao2018')
#                     signal_quality.append(quality)
#             [bad_signals.append(i) for x in signal_quality if x == 'Unacceptable']

#             if len(bad_signals) == 0:
#                 allowed_patients.add_patient(new_Patient)
                
            #removing signals from specific channels that are not of good quality        
            bad_signal_channel = []
            for j in range(0, len(new_Patient.get_channel_names())):
                    cleaned_signal = nk.ecg_clean(signals[:, j])
                    quality = nk.ecg_quality(cleaned_signal, method='zhao2018')
                    if quality == 'Unacceptable':
                        bad_signal_channel.append(j)
                        signals[:, j] = None
            
            new_Patient.set_signal(signals)
            if len(bad_signal_channel) != 6:
                allowed_patients.add_patient(new_Patient)
                
            #removing signals no longer makes sense unless do for every single channel??
#             bad_signal_index_list = [20, 21, 23, 42, 47, 64, 66, 68, 71, 72, 75, 95, 96, 97, 98, 100, 104, 120, 123, 132, 142, 152, 195, 222, 224, 226, 227] # 1 minute signals
            #bad_signal_index_list = [0, 1, 2, 4, 8, 16, 43, 45, 51, 63, 64, 65, 69, 77, 89, 97, 99, 136, 148, 149, 152, 156, 160, 165, 166, 171, 173, 183, 197, 211, 214, 224, 226]#index from neurokit
#             #bad_signal_index_list = [1, 6, 54, 83, 85, 86]#2 minute
            
#             if j not in bad_signal_index_list:
#                  allowed_patients.add_patient(new_Patient)
#             j+=1
            
            
        
    except ValueError as ve:
        None
        #print(f"Error reading record for patient {patients[i]}: {ve}")


  0%|                                                                                          | 0/549 [00:00<?, ?it/s]

Filtering Database


100%|████████████████████████████████████████████████████████████████████████████████| 549/549 [00:58<00:00,  9.34it/s]


In [12]:
no_patients = allowed_patients.count_patients()
print(f"{no_patients} remaining out of 290")

221 remaining out of 290
